In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
#import get_all_tickers as gat
import datetime
import pathlib
import plotly.express as px
import matplotlib.pyplot as plt
import pytz

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [10]:
s = pd.read_html(url)[0]
s = s.drop(columns=["SEC filings","CIK"],inplace=False)

In [14]:
s.loc[:,[col for col in s.columns if not col.count('SEC')]].to_csv("sp500tickers.csv",index=False)